In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

resnet_loc = "./data/resnet_v2_101/resnet_v2_101.ckpt"
log_dir = 'train_log/transfer/all_stages-g4/'
logger.set_logger_dir(log_dir)

[1028 14:44:34 @logger.py:74] Argv: /home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1007/jupyter/kernel-e923b5b8-ab46-403c-80b0-5ee95881bab3.json


In [2]:
config = default
ignore_restore = ['learning_rate', 'global_step']
save_name = "all-stages-max-micro-auc.tfmodel"

#### Split train set and test set

In [10]:
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()

Group numbers:
train: 5014, validation: 0, test: 4726
Image numbers:
train: 22989, validation: 0, test: 21028


#### Gether data to train rnn

In [12]:
config.proportion = {'train': 0.8, 'val':0.2, 'test': 0.0}
config.annotation_number = 30
dm = DataManager.from_dataset(train_set, config)

Group numbers:
train: 3584, validation: 778, test: 0
Image numbers:
train: 16196, validation: 3364, test: 0


In [13]:
config.weight_decay = 0.0
config.dropout_keep_prob = 0.5
config.gamma = 4
config.use_glimpse = True
config.read_time = 5
config.batch_size = 64
config.use_hidden_dense = False

threshold = 0.5
train_data = dm.get_train_stream()
val_data = dm.get_validation_stream()
model = RNN(config, is_finetuning=False)

In [15]:
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (15, 1e-5)]),
                               InfRunner(val_data, [AggregateMetric(config.validation_metrics, threshold)],
                                         [1]),
                               ModelSaver(var_collections='model_variables'),
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=resnet_loc, ignore=ignore_restore),
                           max_epoch=20, tower=[1])
Trainer(train_config).train()

[1028 14:47:49 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 12
[1028 14:47:49 @input_source.py:178] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1028 14:47:49 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:1...
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
[1028 14:47:53 @model_utils.py:47] Model Parameters: 
name                            shape       

[1028 14:47:58 @base.py:212] Creating the session ...
[1028 14:48:03 @base.py:216] Initializing the session ...
[1028 14:48:03 @sessinit.py:116] Restoring checkpoint from ./data/resnet_v2_101/resnet_v2_101.ckpt ...
INFO:tensorflow:Restoring parameters from ./data/resnet_v2_101/resnet_v2_101.ckpt
[1028 14:48:08 @base.py:223] Graph Finalized.
[1028 14:48:08 @param.py:144] After epoch 0, learning_rate will change to 0.00010000
[1028 14:48:08 @concurrency.py:36] Starting EnqueueThread DataParallelInferenceRunner/QueueInput/input_queue ...
[1028 14:48:09 @concurrency.py:36] Starting EnqueueThread QueueInput/input_queue ...
[1028 14:48:09 @base.py:257] Start Epoch 1 ...


100%|##########|56/56[03:28<00:00, 0.36it/s]

[1028 14:51:38 @base.py:267] Epoch 1 (global_step 56) finished, time:208.37 sec.



  0%|          |0/12[00:00<?,?it/s]/home/yangyang/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|12/12[00:21<00:00, 0.56it/s]


...

[1028 15:40:10 @saver.py:89] Model saved to train_log/transfer/all_stages-g4/model-1064.
[1028 15:40:12 @saver.py:158] Model with maximum 'micro_auc' saved.
[1028 15:40:12 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 47.115
[1028 15:40:12 @monitor.py:361] QueueInput/queue_size_1: 0.20894
[1028 15:40:12 @monitor.py:361] coverage: 4.944
[1028 15:40:12 @monitor.py:361] learning_rate: 1e-05
[1028 15:40:12 @monitor.py:361] loss/value: 0.013535
[1028 15:40:12 @monitor.py:361] macro_auc: 0.9188
[1028 15:40:12 @monitor.py:361] macro_f1: 0.25897
[1028 15:40:12 @monitor.py:361] mean_average_precision: 0.61233
[1028 15:40:12 @monitor.py:361] micro_auc: 0.9287
[1028 15:40:12 @monitor.py:361] micro_f1: 0.45713
[1028 15:40:12 @monitor.py:361] one_error: 0.33333
[1028 15:40:12 @monitor.py:361] ranking_loss: 0.069966
[1028 15:40:12 @monitor.py:361] ranking_mean_average_precision: 0.71613
[1028 15:40:12 @monitor.py:361] training_auc: 0.8262
[1028 15:40:12 @group.py:42] Callback

100%|##########|56/56[02:12<00:00, 0.39it/s]

[1028 15:42:25 @base.py:267] Epoch 20 (global_step 1120) finished, time:132.26 sec.



100%|##########|12/12[00:19<00:00, 0.68it/s]


[1028 15:42:45 @saver.py:89] Model saved to train_log/transfer/all_stages-g4/model-1120.
[1028 15:42:47 @saver.py:158] Model with maximum 'micro_auc' saved.
[1028 15:42:47 @monitor.py:361] DataParallelInferenceRunner/QueueInput/queue_size_1: 47.148
[1028 15:42:47 @monitor.py:361] QueueInput/queue_size_1: 0.2985
[1028 15:42:47 @monitor.py:361] coverage: 4.9232
[1028 15:42:47 @monitor.py:361] learning_rate: 1e-05
[1028 15:42:47 @monitor.py:361] loss/value: 0.013615
[1028 15:42:47 @monitor.py:361] macro_auc: 0.91943
[1028 15:42:47 @monitor.py:361] macro_f1: 0.25983
[1028 15:42:47 @monitor.py:361] mean_average_precision: 0.62163
[1028 15:42:47 @monitor.py:361] micro_auc: 0.9297
[1028 15:42:47 @monitor.py:361] micro_f1: 0.45549
[1028 15:42:47 @monitor.py:361] one_error: 0.32552
[1028 15:42:47 @monitor.py:361] ranking_loss: 0.068116
[1028 15:42:47 @monitor.py:361] ranking_mean_average_precision: 0.72452
[1028 15:42:47 @monitor.py:361] training_auc: 0.83049
[1028 15:42:47 @group.py:42] Callba

In [15]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'EMA/QueueInput/queue_size:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/biased:0' shape=() dtype=float32_ref>,
 <tf.Variable 'EMA/QueueInput/queue_size/local_step:0' shape=() dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/conv1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_mean:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/preact/moving_variance:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bottleneck_v2/shortcut/weights:0' shape=(1, 1, 64, 256) dtype=float32_ref>,
 <tf.Variable 'resnet_v2_101/block1/unit_1/bot

In [1]:
a, b = 1, 2